In [1]:
import BlackScholes as bs
import arviz as az
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ImpliedDrift as iD
import time
import variance_curve as vc
import Quintic

In [2]:
dates = np.array(["23_01_23.csv", "24_01_23.csv", "25_01_23.csv", "26_01_23.csv", "27_01_23.csv",
                  "30_01_23.csv", "06_02_23.csv", "13_02_23.csv", "21_02_23.csv"])

spot = np.array(pd.read_csv("spot.csv").Spot).flatten()

moneyness = np.array([80.0,90.0,95.0,97.5,100.0,102.5,105.0,110.0,120.0])

index = 0

parameters = pd.read_csv("quintic_parameters_bayesian2.csv")
rho,H,eps,a0,a1,a3,a5 = np.array(parameters.iloc[index])
a_k = np.array([a0,a1,a3,a5])

data = pd.read_csv(dates[index]); S0 = spot[index];
K = S0 * moneyness / 100

if index:
    maturities = np.array(data.Tenor).flatten()
    IV = np.array(data.drop(columns = ['Date', 'Tenor']))/100.

else:
    maturities = np.array(data['Exp Date']).flatten()
    IV = np.array(data.drop(columns = 'Exp Date'))/100.
    
IV = IV[maturities <= 180/365]
maturities = maturities[maturities <= 180/365]

param = np.array(pd.read_csv("bayesian_data.csv"))

In [3]:
nr = len(maturities); nc = len(K)
N = 15000; n = 180;
T = maturities[-1]; steps = np.ceil(n*maturities/T).astype(int);

np.random.seed(index)
w = np.concatenate((np.zeros([1,N*2]), Quintic.dW(n, N)))
r = iD.r(maturities, index); q = iD.q(maturities, index);

In [4]:
def f(x):
    iv = np.zeros([nr,nc])
    rho,H,eps = x
    P = Quintic.global_reduction(rho, H, eps, T, a_k, S0, K, n, N, w, steps, maturities, index)
    
    for i in range(nr):
        iv[i] = bs.BSImpliedVol(S0, K, maturities[i], r[i], q[i], P[i], Option_type = 1, toll = 1e-5)
    
    return np.mean(abs(IV-iv)/IV)*100, iv

In [5]:
%%timeit
f(np.array([-0.8, 0.1, 0.01]))

1.87 s ± 97.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [6]:
t = time.time()
iv_list = list()
length = len(param); err = np.zeros(length);

for i in range(length):
    err[i], aux = f(param[i])
    iv_list.append(aux)
    if (i+1)%100 == 0:
        print(f"Iteration progress: {(i+1)/length*100: .2f}%")
    
print(f"\nElapsed time: {time.time()-t: .2f}s")

Iteration progress:  1.25%
Iteration progress:  2.50%
Iteration progress:  3.75%
Iteration progress:  5.00%
Iteration progress:  6.25%
Iteration progress:  7.50%
Iteration progress:  8.75%
Iteration progress:  10.00%
Iteration progress:  11.25%
Iteration progress:  12.50%
Iteration progress:  13.75%
Iteration progress:  15.00%
Iteration progress:  16.25%
Iteration progress:  17.50%
Iteration progress:  18.75%
Iteration progress:  20.00%
Iteration progress:  21.25%
Iteration progress:  22.50%
Iteration progress:  23.75%
Iteration progress:  25.00%
Iteration progress:  26.25%
Iteration progress:  27.50%
Iteration progress:  28.75%
Iteration progress:  30.00%
Iteration progress:  31.25%
Iteration progress:  32.50%
Iteration progress:  33.75%
Iteration progress:  35.00%
Iteration progress:  36.25%
Iteration progress:  37.50%
Iteration progress:  38.75%
Iteration progress:  40.00%
Iteration progress:  41.25%
Iteration progress:  42.50%
Iteration progress:  43.75%
Iteration progress:  45.00%

In [7]:
err.min()

2.084712018867645

In [8]:
err.max()

8.784280453841928

In [9]:
err.mean()

3.6686599798943527

In [10]:
err.std()

0.9051242894160663

In [11]:
err.var()

0.8192499792909388

In [12]:
param[np.argmin(err)]

array([-0.90743533,  0.00125434,  0.03239357])

In [13]:
df = pd.DataFrame(err, columns = ["Error"])
df.to_csv("grid_error.csv", index = False)

In [14]:
iv = np.zeros([length, len(iv_list[0].flatten())])

for i in range(length):
    iv[i] = iv_list[i].flatten()

In [15]:
db = pd.DataFrame(iv)
db.to_csv("grid_iv.csv", index = False)